In [11]:
#load saved models and data

class TopPop(AlgoBase):
    def __init__(self, top_items_list=None):
        AlgoBase.__init__(self)
        self.top_items = top_items_list if top_items_list else []
    
    def fit(self, trainset):
        self.trainset = trainset
        return self
    
    def estimate(self, u, i):
        if self.trainset.to_raw_iid(i) in self.top_items:
            return 5.0
        else:
            return 1.0

# Load the cleaned data
train_df = pd.read_parquet('cleaned_train.parquet')
test_df = pd.read_parquet('cleaned_test.parquet')

# Load the saved models
with open('best_knn_model.pkl', 'rb') as f:
    knn_model = pickle.load(f)

with open('best_svd_model.pkl', 'rb') as f:
    svd_model = pickle.load(f)

with open('top_pop_model.pkl', 'rb') as f:
    top_pop_model = pickle.load(f)

with open('trainset.pkl', 'rb') as f:
    trainset = pickle.load(f)

print("Models and data loaded successfully!")
print(f"Train set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")

Models and data loaded successfully!
Train set size: 26580
Test set size: 6645


In [8]:
import pandas as pd
import numpy as np
import pickle
from surprise import accuracy, AlgoBase
import warnings
warnings.filterwarnings('ignore')

print("everything imported")


everything imported
